# Retrieval Augmented Generation (RAG) アプリを構築する: パート1

LLMによって可能になる最も強力なアプリケーションの1つは、洗練された質問応答 (Q&A) チャットボットです。これらは特定のソース情報に関する質問に答えるアプリケーションです。これらのアプリケーションは、リトリーバル強化生成 ([RAG](https://python.langchain.com/docs/concepts/rag/)) として知られる技術を使用します。

これは複数パートのチュートリアルです:

- [パート1](https://python.langchain.com/docs/tutorials/rag/) (このガイド) では、RAGを紹介し、最小限の実装を説明します。
- [パート2](https://python.langchain.com/docs/tutorials/qa_chat_history/)では、会話スタイルのインタラクションやマルチステップのリトリーバルプロセスに対応するために実装を拡張します。

このチュートリアルでは、テキストデータソース上でシンプルなQ&Aアプリケーションを構築する方法を示します。途中で、典型的なQ&Aアーキテクチャについて説明し、より高度なQ&A技術のための追加リソースを強調します。また、LangSmithがアプリケーションのトレースと理解にどのように役立つかを見ていきます。アプリケーションが複雑になるにつれて、LangSmithはますます役立つようになります。

基本的なリトリーバルにすでに精通している場合は、[さまざまなリトリーバル技術の概要](https://python.langchain.com/docs/concepts/retrieval/)にも興味があるかもしれません。

注: ここでは非構造化データのQ&Aに焦点を当てています。構造化データに対するRAGに興味がある場合は、[SQLデータに対する質問応答](https://python.langchain.com/docs/tutorials/sql_qa/)に関するチュートリアルをご覧ください。

[Build a Retrieval Augmented Generation \(RAG\) App: Part 1 \| 🦜️🔗 LangChain](https://python.langchain.com/docs/tutorials/rag/)

## 概要

典型的なRAGアプリケーションには2つの主要なコンポーネントがあります:

- **インデックス作成:** ソースからデータを取り込み、インデックスを作成するパイプライン。*これは通常オフラインで行われます。*
- **リトリーバルと生成:** 実際のRAGチェーンで、実行時にユーザーのクエリを受け取り、インデックスから関連データを取得し、それをモデルに渡します。

注: このチュートリアルのインデックス作成部分は、[主にセマンティック検索のチュートリアル](https://python.langchain.com/docs/tutorials/retrievers/)に従います。

生データから回答までの最も一般的な完全なシーケンスは次のとおりです:

### インデックス作成

1. **ロード:** まず、データをロードする必要があります。これは[ドキュメントローダー](https://python.langchain.com/docs/concepts/document_loaders/)を使用して行います。
1. **分割:** [テキストスプリッター](https://python.langchain.com/docs/concepts/text_splitters/)は、大きな`Documents`を小さなチャンクに分割します。これは、データのインデックス作成やモデルに渡す際に役立ちます。大きなチャンクは検索が難しく、モデルの有限のコンテキストウィンドウに収まりません。
1. **保存:** 分割したデータを保存し、後で検索できるようにインデックスを作成する場所が必要です。これは通常、[ベクトルストア](https://python.langchain.com/docs/concepts/vectorstores/)と[エンベディング](https://python.langchain.com/docs/concepts/embedding_models/)モデルを使用して行います。

![](https://python.langchain.com/assets/images/rag_indexing-8160f90a90a33253d0154659cf7d453f.png)

### リトリーバルと生成

1. **リトリーバル:** ユーザー入力を受け取り、リトリーバーを使用してストレージから関連するスプリットを取得します。
1. **生成:** [チャットモデル](https://python.langchain.com/docs/concepts/chat_models/) / [LLM](https://python.langchain.com/docs/concepts/text_llms/)が、質問と取得したデータを含むプロンプトを使用して回答を生成します。

![](https://python.langchain.com/assets/images/rag_retrieval_generation-1046a4668d6bb08786ef73c56d4f228a.png)

データのインデックス作成が完了したら、[LangGraph](https://langchain-ai.github.io/langgraph/)をオーケストレーションフレームワークとして使用し、リトリーバルと生成のステップを実装します。

## セットアップ

In [0]:
%pip install --quiet --upgrade langchain-text-splitters langchain-community langgraph langchain[openai] mlflow bs4
%restart_python

In [0]:
import mlflow

# MLflow Tracingの有効化
mlflow.langchain.autolog()

In [0]:
import os
os.environ["OPENAI_API_KEY"] = dbutils.secrets.get("demo-token-takaaki.yayoi", "openai_api_key")

## コンポーネント

LangChainの統合スイートから3つのコンポーネントを選択する必要があります。

[chat model](https://python.langchain.com/docs/integrations/chat/)を選択します。

In [0]:
from langchain.chat_models import init_chat_model

llm = init_chat_model("gpt-4o-mini", model_provider="openai")

[Embeddings model](https://python.langchain.com/docs/integrations/text_embedding/)を選択します。

In [0]:
from langchain_openai import OpenAIEmbeddings

embeddings = OpenAIEmbeddings(model="text-embedding-3-large")

[Vector store](https://python.langchain.com/docs/integrations/vectorstores/)を選択します。

In [0]:
from langchain_core.vectorstores import InMemoryVectorStore

vector_store = InMemoryVectorStore(embeddings)

## プレビュー

このガイドでは、ウェブサイトのコンテンツに関する質問に答えるアプリを作成します。使用する特定のウェブサイトは、私による[はじめてのDatabricks](https://qiita.com/taka_yayoi/items/8dc72d083edb879a5e5d)のブログ投稿であり、投稿の内容に関する質問をすることができます。

これを約50行のコードでシンプルなインデックス作成パイプラインとRAGチェーンを作成して行うことができます。

In [0]:
import bs4
from langchain import hub
from langchain_community.document_loaders import WebBaseLoader
from langchain_core.documents import Document
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langgraph.graph import START, StateGraph
from typing_extensions import List, TypedDict

# ブログの内容をロードしてチャンクに分割
loader = WebBaseLoader(
    web_paths=("https://qiita.com/taka_yayoi/items/8dc72d083edb879a5e5d",),
    bs_kwargs=dict(
        parse_only=bs4.SoupStrainer(
            class_=("p-items_main")
        )
    ),
)
docs = loader.load()

text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
all_splits = text_splitter.split_documents(docs)

# チャンクをインデックス化
_ = vector_store.add_documents(documents=all_splits)

# 質問応答のためのプロンプトを定義
prompt = hub.pull("rlm/rag-prompt")


# アプリケーションの状態を定義
class State(TypedDict):
    question: str
    context: List[Document]
    answer: str


# アプリケーションのステップを定義
def retrieve(state: State):
    retrieved_docs = vector_store.similarity_search(state["question"])
    return {"context": retrieved_docs}


def generate(state: State):
    docs_content = "\n\n".join(doc.page_content for doc in state["context"])
    messages = prompt.invoke({"question": state["question"], "context": docs_content})
    response = llm.invoke(messages)
    return {"answer": response.content}


# アプリケーションをコンパイルしてテスト
graph_builder = StateGraph(State).add_sequence([retrieve, generate])
graph_builder.add_edge(START, "retrieve")
graph = graph_builder.compile()

In [0]:
docs

In [0]:
from IPython.display import Image, display
display(Image(graph.get_graph().draw_mermaid_png()))

In [0]:
response = graph.invoke({"question": "DatabricksとJupyter Notebookの違いは？"})
print(response["answer"])